## Milestone 2
It has been three months since your successful analysis. While casually browsing the analytics platform for another project, you encounter a previously unseen variable that the client had, for some reason, not pulled. It appears that the advertising campaign had actually been run as an A/B test.

New variable: 
* ASSIGNED_DISPLAY_AD – flag indicating whether the customer was assigned to receive ad


1. The estimated effect of their display advertising, based on the random assignment.
2. The estimated ROI of their display advertising, based on the random assignment.
3. Interpretation of these estimates and what they mean for the client’s marketing investment.
4. If these estimates are different from the Milestone 1 estimates, how would you explain these differences to the client?

In [1]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, f1_score
import statsmodels.api as sm

In [2]:
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [3]:
m1_dir = '/Users/tsebaotshewarega/Documents/Third year/Fall 2023/QTM 385/eCommerce/eCommerce_class/ecomm_performance_data/milestone-1/ecomm_rev_2020.csv'
m2_dir = '/Users/tsebaotshewarega/Documents/Third year/Fall 2023/QTM 385/eCommerce/eCommerce_class/ecomm_performance_data/milestone-2/ecomm_rev_2020_incl_assn.csv'
m3_dir = '/Users/tsebaotshewarega/Documents/Third year/Fall 2023/QTM 385/eCommerce/eCommerce_class/ecomm_performance_data/milestone-3/ecomm_rev_2020_omnipotent.csv'

m1_df = pd.read_csv(m1_dir)
m2_df = pd.read_csv(m2_dir)
m3_df = pd.read_csv(m3_dir)


# Create an instance of the OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Fit and transform the data
one_hot_encoded = encoder.fit_transform(m1_df[['ESTIMATED_INCOME_DECILE']])

# Convert the result to a DataFrame for better visualization
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(['ESTIMATED_INCOME_DECILE']))

# Concatenate the one-hot-encoded DataFrame with the original DataFrame
m1_df = pd.concat([m1_df, one_hot_df], axis=1)
m2_df = pd.concat([m2_df, one_hot_df], axis=1)
m3_df = pd.concat([m3_df, one_hot_df], axis=1)

In [4]:
m2_df

,CUSTID,CART_TOTAL,PREVIOUS_CHECKOUTS,PAGE_VIEWS,ESTIMATED_INCOME_DECILE,PRODUCT_VIEWS,ASSIGNED_DISPLAY_AD,ATTRIB_DISPLAY_AD,ESTIMATED_INCOME_DECILE_1,ESTIMATED_INCOME_DECILE_2,ESTIMATED_INCOME_DECILE_3,ESTIMATED_INCOME_DECILE_4,ESTIMATED_INCOME_DECILE_5,ESTIMATED_INCOME_DECILE_6,ESTIMATED_INCOME_DECILE_7,ESTIMATED_INCOME_DECILE_8,ESTIMATED_INCOME_DECILE_9,ESTIMATED_INCOME_DECILE_10
0,3218c960b1144725db928e476eddf136,17.00,0,5,1,9,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,342ce76c22408d0e9cdf13afa6934be8,9.64,0,7,4,9,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,625ebd9228821291d23c6e2a7cf7f05e,39.57,3,4,5,10,1,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,1941f3295dae57c0b5deeedfb0cedb0c,35.95,5,9,9,10,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,dd50c9de777380b997e1769d61c9829e,42.23,0,4,7,10,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2870,eb00d664738ac99c9c1e65f124effcb0,39.55,0,7,1,11,1,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2871,1f0b2a538fb209e8111f4724aeb59c43,59.93,0,4,1,10,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2872,bdd43dd162d0b5788fd30c43cb042df4,52.99,0,3,10,12,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2873,73895496a872752af0e5eadbbca5d554,75.99,7,5,5,10,0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [5]:
freq_table = pd.crosstab(m2_df['ASSIGNED_DISPLAY_AD'], m2_df['ATTRIB_DISPLAY_AD']) 
freq_table

ATTRIB_DISPLAY_AD,0,1
ASSIGNED_DISPLAY_AD,,
0,1430,0
1,706,739


From the table above we can see, all customers that weren't assigned the treatment (ad) didn't see the display ad, but 706 customers were assigned the see the ad and didn't. 
To be able to leaverage the __random__ treatment assignemnt, we need to estimate the Intent to treat effect. 

In [6]:
round(m2_df.groupby('ASSIGNED_DISPLAY_AD')['CART_TOTAL'].describe(),3)

,count,mean,std,min,25%,50%,75%,max
ASSIGNED_DISPLAY_AD,,,,,,,,
0,1430.0,49.806,21.533,0.0,35.218,49.345,63.982,115.38
1,1445.0,45.691,19.093,0.0,34.670,44.900,55.880,114.75


In [7]:
round(m2_df.groupby('ASSIGNED_DISPLAY_AD').mean(),3)

,CART_TOTAL,PREVIOUS_CHECKOUTS,PAGE_VIEWS,ESTIMATED_INCOME_DECILE,PRODUCT_VIEWS,ATTRIB_DISPLAY_AD,ESTIMATED_INCOME_DECILE_1,ESTIMATED_INCOME_DECILE_2,ESTIMATED_INCOME_DECILE_3,ESTIMATED_INCOME_DECILE_4,ESTIMATED_INCOME_DECILE_5,ESTIMATED_INCOME_DECILE_6,ESTIMATED_INCOME_DECILE_7,ESTIMATED_INCOME_DECILE_8,ESTIMATED_INCOME_DECILE_9,ESTIMATED_INCOME_DECILE_10
ASSIGNED_DISPLAY_AD,,,,,,,,,,,,,,,,
0,49.806,0.840,5.027,5.424,9.986,0.000,0.099,0.097,0.11,0.1,0.106,0.109,0.099,0.094,0.086,0.099
1,45.691,0.933,4.992,5.566,9.984,0.511,0.102,0.103,0.09,0.1,0.095,0.091,0.100,0.105,0.113,0.100


This further supports the random assignment, once grouped by treatment assignment, the two groups seem to be balanced by covariates. 

In [8]:
ATE_naive2 = m2_df.loc[m2_df["ASSIGNED_DISPLAY_AD"]==1]["CART_TOTAL"].mean() - m2_df.loc[m2_df["ASSIGNED_DISPLAY_AD"]==0]["CART_TOTAL"].mean()
print (f"The naive estimated effect of the dispaly ads is: {ATE_naive2}")

The naive estimated effect of the dispaly ads is: -4.115816996152596


Calculating the CATE would be a better estimate for this, because we know we have non-compliers. This would estimate the average treatment effect for compliers only. 

In [9]:
# the compliers 
comp = m2_df[m2_df['ASSIGNED_DISPLAY_AD']==m2_df['ATTRIB_DISPLAY_AD']]
CATE = comp.loc[m2_df["ASSIGNED_DISPLAY_AD"]==1]["CART_TOTAL"].mean() - comp.loc[m2_df["ASSIGNED_DISPLAY_AD"]==0]["CART_TOTAL"].mean()
print (f"The CATE of the dispaly ads is: {CATE}")

The CATE of the dispaly ads is: 9.071396623674104


### Two Stage Least Squares (2SLS)

In [10]:
# First Stage:
from statsmodels.regression.linear_model import OLS

features = m2_df[['PREVIOUS_CHECKOUTS', 'PAGE_VIEWS', 'ESTIMATED_INCOME_DECILE', 'PRODUCT_VIEWS', 
                  'ESTIMATED_INCOME_DECILE_1', 'ESTIMATED_INCOME_DECILE_2', 'ESTIMATED_INCOME_DECILE_3',
                    'ESTIMATED_INCOME_DECILE_4', 'ESTIMATED_INCOME_DECILE_5', 'ESTIMATED_INCOME_DECILE_6', 
                    'ESTIMATED_INCOME_DECILE_7', 'ESTIMATED_INCOME_DECILE_8', 'ESTIMATED_INCOME_DECILE_9', 
                    'ESTIMATED_INCOME_DECILE_10']]
endog_feature = m2_df['ATTRIB_DISPLAY_AD']

instr_plus_controls = m2_df[['PREVIOUS_CHECKOUTS', 'PAGE_VIEWS', 'ESTIMATED_INCOME_DECILE', 'PRODUCT_VIEWS', 
                  'ESTIMATED_INCOME_DECILE_1', 'ESTIMATED_INCOME_DECILE_2', 'ESTIMATED_INCOME_DECILE_3',
                    'ESTIMATED_INCOME_DECILE_4', 'ESTIMATED_INCOME_DECILE_5', 'ESTIMATED_INCOME_DECILE_6', 
                    'ESTIMATED_INCOME_DECILE_7', 'ESTIMATED_INCOME_DECILE_8', 'ESTIMATED_INCOME_DECILE_9', 
                    'ESTIMATED_INCOME_DECILE_10', 'ASSIGNED_DISPLAY_AD']]

instr_plus_controls_plus_constant = sm.add_constant(instr_plus_controls)

first_stage = OLS(endog_feature, instr_plus_controls_plus_constant,missing='drop').fit()
first_stage.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      ATTRIB_DISPLAY_AD   R-squared:                       0.403
Model:                            OLS   Adj. R-squared:                  0.400
Method:                 Least Squares   F-statistic:                     148.6
Date:                Sat, 04 Jan 2025   Prob (F-statistic):          1.54e-308
Time:                        22:39:44   Log-Likelihood:                -957.79
No. Observations:                2875   AIC:                             1944.
Df Residuals:                    2861   BIC:                             2027.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                         -0.6599      0.044    -15.106      0.000      -0.746      -0.574
PREVIOUS_CHECKOUTS             0.0171      0.005      3.645      0.000       0.008       0.026
PAGE_VIEWS                     0.0038      0.003      1.302      0.193      -0.002       0.009
ESTIMATED_INCOME_DECILE       -0.0262      0.003     -7.934      0.000      -0.033      -0.020
PRODUCT_VIEWS                  0.0839      0.006     13.667      0.000       0.072       0.096
ESTIMATED_INCOME_DECILE_1     -0.2843      0.024    -11.991      0.000      -0.331      -0.238
ESTIMATED_INCOME_DECILE_2     -0.1840      0.023     -8.129      0.000      -0.228      -0.140
ESTIMATED_INCOME_DECILE_3     -0.1741      0.021     -8.129      0.000      -0.216      -0.132
ESTIMATED_INCOME_DECILE_4     -0.1183      0.021     -5.764      0.000      -0.159      -0.078
ESTIMATED_INCOME_DECILE_5     -0.0925      0.020     -4.672      0.000      -0.131      -0.054
ESTIMATED_INCOME_DECILE_6     -0.0620      0.019     -3.240      0.001      -0.100      -0.024
ESTIMATED_INCOME_DECILE_7     -0.0094      0.019     -0.501      0.616      -0.046       0.027
ESTIMATED_INCOME_DECILE_8      0.0260      0.018      1.416      0.157      -0.010       0.062
ESTIMATED_INCOME_DECILE_9      0.0730      0.018      3.958      0.000       0.037       0.109
ESTIMATED_INCOME_DECILE_10     0.1657      0.018      9.122      0.000       0.130       0.201
ASSIGNED_DISPLAY_AD            0.5077      0.013     40.101      0.000       0.483       0.533
==============================================================================
Omnibus:                      322.038   Durbin-Watson:                   1.974
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               89.798
Skew:                          -0.046   Prob(JB):                     3.17e-20
Kurtosis:                       2.139   Cond. No.                     2.19e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 9.57e-28. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [11]:
## second stage 
m2_df['VIEW_PROJ'] = first_stage.fittedvalues

proj_features = m2_df[['PREVIOUS_CHECKOUTS', 'PAGE_VIEWS', 'ESTIMATED_INCOME_DECILE', 'PRODUCT_VIEWS', 
                  'ESTIMATED_INCOME_DECILE_1', 'ESTIMATED_INCOME_DECILE_2', 'ESTIMATED_INCOME_DECILE_3',
                    'ESTIMATED_INCOME_DECILE_4', 'ESTIMATED_INCOME_DECILE_5', 'ESTIMATED_INCOME_DECILE_6', 
                    'ESTIMATED_INCOME_DECILE_7', 'ESTIMATED_INCOME_DECILE_8', 'ESTIMATED_INCOME_DECILE_9', 
                    'ESTIMATED_INCOME_DECILE_10', 'VIEW_PROJ']]
outcome = m2_df['CART_TOTAL']
proj_feature_plus_constant = sm.add_constant(proj_features)

second_stage = OLS(outcome, proj_feature_plus_constant,missing='drop').fit()
second_stage.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             CART_TOTAL   R-squared:                       0.248
Model:                            OLS   Adj. R-squared:                  0.245
Method:                 Least Squares   F-statistic:                     72.64
Date:                Sat, 04 Jan 2025   Prob (F-statistic):          2.36e-166
Time:                        22:39:46   Log-Likelihood:                -12345.
No. Observations:                2875   AIC:                         2.472e+04
Df Residuals:                    2861   BIC:                         2.480e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                        -35.2359      2.371    -14.861      0.000     -39.885     -30.587
PREVIOUS_CHECKOUTS             1.1882      0.249      4.778      0.000       0.701       1.676
PAGE_VIEWS                     0.4113      0.152      2.704      0.007       0.113       0.710
ESTIMATED_INCOME_DECILE       -0.6310      0.173     -3.641      0.000      -0.971      -0.291
PRODUCT_VIEWS                  8.9226      0.340     26.223      0.000       8.255       9.590
ESTIMATED_INCOME_DECILE_1    -16.4518      1.268    -12.972      0.000     -18.939     -13.965
ESTIMATED_INCOME_DECILE_2     -9.2179      1.194     -7.723      0.000     -11.558      -6.877
ESTIMATED_INCOME_DECILE_3     -9.6113      1.141     -8.424      0.000     -11.848      -7.374
ESTIMATED_INCOME_DECILE_4     -6.0835      1.082     -5.624      0.000      -8.204      -3.962
ESTIMATED_INCOME_DECILE_5     -3.6804      1.045     -3.521      0.000      -5.730      -1.631
ESTIMATED_INCOME_DECILE_6     -2.6972      1.009     -2.672      0.008      -4.676      -0.718
ESTIMATED_INCOME_DECILE_7     -1.7843      0.980     -1.820      0.069      -3.706       0.138
ESTIMATED_INCOME_DECILE_8      1.8169      0.963      1.887      0.059      -0.071       3.705
ESTIMATED_INCOME_DECILE_9      4.7706      0.974      4.899      0.000       2.861       6.680
ESTIMATED_INCOME_DECILE_10     7.7029      0.966      7.976      0.000       5.809       9.597
VIEW_PROJ                     -8.6835      1.309     -6.633      0.000     -11.250      -6.117
==============================================================================
Omnibus:                        1.964   Durbin-Watson:                   1.920
Prob(Omnibus):                  0.375   Jarque-Bera (JB):                1.942
Skew:                           0.030   Prob(JB):                        0.379
Kurtosis:                       3.112   Cond. No.                     2.03e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.11e-27. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [18]:
second_stage.params['VIEW_PROJ']

-8.683455853994854

In [22]:
print(f"The estimated effect of the diplay ad that was randomly assigned using 2SLS: {round(second_stage.params['VIEW_PROJ'], 3)}$")
ROI = ((739* second_stage.params['VIEW_PROJ']) - 10000) /10000 *100
print(f"ROI is {round(ROI, 2)}%")

The estimated effect of the diplay ad that was randomly assigned using 2SLS: -8.683$
ROI is -164.17%
